In [5]:
from bs4 import BeautifulSoup as bs
import requests
import ssl
import re

import numpy as np
import pandas as pd

In [11]:
url = "https://slashdot.org/"

headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9,es;q=0.8',
    #'cache-control': 'max-age=0',
    'dnt': '1',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'cross-site',
    'sec-fetch-user': '?1',
    'sec-gpc': '1',
    'upgrade-insecure-requests': '1',
    'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.101 Safari/537.36'
    }

ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

r = requests.get(url, headers=headers)
soup = bs(r.text, 'html.parser')

In [79]:
# Title, link, rating
tit = [t.get_text() for t in soup.select('h2 span a')]

# Title of post
title = [j for i in soup.find_all('span', class_='story-title') for j in i.find('a')]

# Date/time of post
from datetime import datetime

dated = [d.get_text() for d in soup.select('span.story-byline time')]
date = [re.sub('on|@', '', x).strip() for x in dated]
dt = [datetime.strptime(d, "%A %B %d, %Y %I:%M%p") for d in date]

# External link to post
elink = [l.text.strip() for l in soup.select('h2 span span')]

# Votes on post
votes = [v.text.strip() for v in soup.find_all('span', class_='comment-bubble')]

# Category of post
classification = [b.get('alt') for b in soup.find_all('img')]

# User who made the post
user = [u.get_text() for u in soup.select('span.story-byline a')]

# Popularity of post (ratings? red?)
onclick = [re.findall("'([a-zA-Z0-9,\s]*)'", prop['onclick']) for prop in soup.find_all("span", attrs={"alt":"Popularity"})]
pop = [p[1] for p in onclick]

In [ ]:
[i for i in classification if len(i) != 0]

In [ ]:
pd.DataFrame({'title': title, 'date': dt, 'exlink': elink, 'category': classification, 'user': user, 'popular': pop})